In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Glucocorticoid_Sensitivity/GSE50012'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Comparison of cellular and transcriptional responses to 1,25-dihydroxyvitamin D3 and glucocorticoids in peripheral blood mononuclear cells"
!Series_summary	"Glucocorticoids (GC) and 1,25-dihydroxyvitamin D3 (1,25(OH)2 D3) are steroid hormones with anti-inflammatory properties with enhanced effects when combined. We previously showed that transcriptional response to GCs was correlated with inter-individual and inter-ethnic cellular response. Here, we profiled cellular and transcriptional responses to 1,25(OH)2 D3 from the same donors. We studied cellular response to combined treatment with GCs and 1,25(OH)2 D3 in a subset of individuals least responsive to GCs. We found that combination treatment had significantly greater inhibition of proliferation than with either steroid hormone alone. Overlapping differentially expressed (DE) genes between the two hormones were enriched for adaptive and innate immune processes. Non-overlapping differentially ex

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Determine if gene expression data is available
is_gene_available = True  # based on the dataset description, it seems to include gene expression data

# Step 2: Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Checking the availability of 'Glucocorticoid_Sensitivity', 'age', and 'gender'

# "Glucocorticoid_Sensitivity" identified in key 3
if len(set(['89.43486', '95.88507', '95.22036', '92.86704', '93.71633', '96.76962', 
           '88.55031', '90.09957', '94.17097', '86.97089', '98.34904', '91.14896'])) > 1:
    trait_row = 3

# "age" identified in key 5 and 6 (values are fairly continuous in key 6)
if len(set(['44.15342', '24.72329', '32.37808', '20.38082', '21.2411', '22.54247', 
            '26.13973', '21.5616', '21.9863', '26.76712', '23.59452', '23.47945'])) > 1:
    age_row = 6

# "gender" identified in key 4
if len(set(['female', 'male'])) == 2:
    gender_row = 4

# 2.3 Data Type Conversion
def convert_trait(value):
    try:
        value = str(value)
        return float(value.split(':')[-1].strip())
    except (ValueError, AttributeError):
        return None

def convert_age(value):
    try:
        value = str(value)
        return float(value.split(':')[-1].strip())
    except (ValueError, AttributeError):
        return None

def convert_gender(value):
    try:
        value = str(value)
        val = value.split(':')[-1].strip().lower()
        if val == 'female':
            return 0
        elif val == 'male':
            return 1
    except (ValueError, AttributeError):
        return None
    return None

# Step 3: Save cohort information
save_cohort_info('GSE50012', './preprocessed/Glucocorticoid_Sensitivity/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Glucocorticoid_Sensitivity', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Glucocorticoid_Sensitivity/trait_data/GSE50012.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM832137': [89.43486, 44.15342, nan], 'GSM832138': [89.43486, 44.15342, nan], 'GSM832139': [89.43486, 44.15342, nan], 'GSM832140': [89.43486, 44.15342, nan], 'GSM832141': [95.88507, 24.72329, nan], 'GSM832142': [95.88507, 24.72329, nan], 'GSM832143': [95.88507, 24.72329, nan], 'GSM832144': [95.88507, 24.72329, nan], 'GSM832145': [95.22036, 32.37808, nan], 'GSM832146': [95.22036, 32.37808, nan], 'GSM832147': [95.22036, 32.37808, nan], 'GSM832148': [95.22036, 32.37808, nan], 'GSM832149': [92.86704, 20.38082, nan], 'GSM832150': [92.86704, 20.38082, nan], 'GSM832151': [92.86704, 20.38082, nan], 'GSM832152': [92.86704, 20.38082, nan], 'GSM832153': [93.71633, 21.2411, nan], 'GSM832154': [93.71633, 21.2411, nan], 'GSM832155': [93.71633, 21.2411, nan], 'GSM832156': [93.71633, 21.2411, nan], 'GSM832157': [96.76962, 22.54247, nan], 'GSM832158': [96.76962, 22.54247, nan], 'GSM832159': [96.76962, 22.54247, nan], 'GSM832160': [96.76962, 22.54247, nan], 'GSM832161': [88.55031, 26.13973, nan], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifier and gene symbol
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# Adding assertions to check if the keys exist in the gene annotation columns
assert identifier_key in gene_annotation.columns, f"Identifier key '{identifier_key}' not found in gene annotation columns."
assert gene_symbol_key in gene_annotation.columns, f"Gene symbol key '{gene_symbol_key}' not found in gene annotation columns."

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
   
# 3. Apply the mapping to the gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)
